In [84]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [85]:
import torch
import pandas as pd

## Baseline Model

In [86]:
dataset = torch.load('hexagon_812_final.pt')

In [87]:
for i in range(len(dataset)):
  a=pd.DataFrame(dataset[i].x.numpy()).drop([0,1,4,5],axis=1).to_numpy() #drop id
  dataset[i].x=torch.from_numpy(a)

In [88]:
import random
random.shuffle(dataset)

In [89]:
train_dataset = dataset[:650]
test_dataset = dataset[650:]

In [90]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [91]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(2, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(2, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [93]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.03)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x.float(), data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x.float(), data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5015, Test Acc: 0.4938
Epoch: 002, Train Acc: 0.5015, Test Acc: 0.4938
Epoch: 003, Train Acc: 0.5015, Test Acc: 0.4938
Epoch: 004, Train Acc: 0.4985, Test Acc: 0.5062
Epoch: 005, Train Acc: 0.5015, Test Acc: 0.4938
Epoch: 006, Train Acc: 0.4985, Test Acc: 0.5062
Epoch: 007, Train Acc: 0.5015, Test Acc: 0.4938
Epoch: 008, Train Acc: 0.5015, Test Acc: 0.4938
Epoch: 009, Train Acc: 0.5015, Test Acc: 0.4938
Epoch: 010, Train Acc: 0.5015, Test Acc: 0.4938
Epoch: 011, Train Acc: 0.4985, Test Acc: 0.5062
Epoch: 012, Train Acc: 0.5015, Test Acc: 0.4938
Epoch: 013, Train Acc: 0.4985, Test Acc: 0.5062
Epoch: 014, Train Acc: 0.5015, Test Acc: 0.4938
Epoch: 015, Train Acc: 0.5015, Test Acc: 0.4938
Epoch: 016, Train Acc: 0.4985, Test Acc: 0.5062
Epoch: 017, Train Acc: 0.4985, Test Acc: 0.5062
Epoch: 018, Train Acc: 0.5015, Test Acc: 0.4938
Epoch: 019, Train Acc: 0.4985, Test Acc: 0.5062
Epoch: 020, Train Acc: 0.4985, Test Acc: 0.5062
Epoch: 021, Train Acc: 0.4985, Test Acc:

## Triangles

In [94]:
dataset = torch.load('hexagon_812_final.pt')

In [95]:
for i in range(len(dataset)):
  a=pd.DataFrame(dataset[i].x.numpy()).drop([0,1,5],axis=1).to_numpy() #drop id,time,number of hexagons
  dataset[i].x=torch.from_numpy(a)

In [96]:
import random
random.shuffle(dataset)

In [97]:
train_dataset = dataset[:650]
test_dataset = dataset[650:]

In [98]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [99]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(3, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(3, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [100]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.03)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x.float(), data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x.float(), data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5154, Test Acc: 0.4383
Epoch: 002, Train Acc: 0.5092, Test Acc: 0.4568
Epoch: 003, Train Acc: 0.4846, Test Acc: 0.5617
Epoch: 004, Train Acc: 0.5154, Test Acc: 0.4383
Epoch: 005, Train Acc: 0.5154, Test Acc: 0.4383
Epoch: 006, Train Acc: 0.5154, Test Acc: 0.4383
Epoch: 007, Train Acc: 0.4846, Test Acc: 0.5617
Epoch: 008, Train Acc: 0.5154, Test Acc: 0.4383
Epoch: 009, Train Acc: 0.5154, Test Acc: 0.4383
Epoch: 010, Train Acc: 0.5154, Test Acc: 0.4383
Epoch: 011, Train Acc: 0.5154, Test Acc: 0.4383
Epoch: 012, Train Acc: 0.4846, Test Acc: 0.5617
Epoch: 013, Train Acc: 0.5154, Test Acc: 0.4383
Epoch: 014, Train Acc: 0.4846, Test Acc: 0.5617
Epoch: 015, Train Acc: 0.4846, Test Acc: 0.5617
Epoch: 016, Train Acc: 0.5154, Test Acc: 0.4383
Epoch: 017, Train Acc: 0.5154, Test Acc: 0.4383
Epoch: 018, Train Acc: 0.5154, Test Acc: 0.4383
Epoch: 019, Train Acc: 0.5154, Test Acc: 0.4383
Epoch: 020, Train Acc: 0.5154, Test Acc: 0.4383
Epoch: 021, Train Acc: 0.5154, Test Acc:

## Hexagons

In [101]:
dataset = torch.load('hexagon_812_final.pt')

In [102]:
for i in range(len(dataset)):
  a=pd.DataFrame(dataset[i].x.numpy()).drop([0,1,4],axis=1).to_numpy() #drop id,time,number of triangles
  dataset[i].x=torch.from_numpy(a)

In [103]:
import random
random.shuffle(dataset)

In [104]:
train_dataset = dataset[:650]
test_dataset = dataset[650:]

In [105]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [106]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(3, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(3, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [107]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.03)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x.float(), data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x.float(), data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4862, Test Acc: 0.5556
Epoch: 002, Train Acc: 0.5138, Test Acc: 0.4444
Epoch: 003, Train Acc: 0.4862, Test Acc: 0.5556
Epoch: 004, Train Acc: 0.5138, Test Acc: 0.4444
Epoch: 005, Train Acc: 0.4862, Test Acc: 0.5556
Epoch: 006, Train Acc: 0.5138, Test Acc: 0.4444
Epoch: 007, Train Acc: 0.5138, Test Acc: 0.4444
Epoch: 008, Train Acc: 0.5138, Test Acc: 0.4444
Epoch: 009, Train Acc: 0.5138, Test Acc: 0.4444
Epoch: 010, Train Acc: 0.5138, Test Acc: 0.4444
Epoch: 011, Train Acc: 0.5138, Test Acc: 0.4444
Epoch: 012, Train Acc: 0.5138, Test Acc: 0.4444
Epoch: 013, Train Acc: 0.4862, Test Acc: 0.5556
Epoch: 014, Train Acc: 0.4862, Test Acc: 0.5556
Epoch: 015, Train Acc: 0.5138, Test Acc: 0.4444
Epoch: 016, Train Acc: 0.5138, Test Acc: 0.4444
Epoch: 017, Train Acc: 0.5138, Test Acc: 0.4444
Epoch: 018, Train Acc: 0.5138, Test Acc: 0.4444
Epoch: 019, Train Acc: 0.5138, Test Acc: 0.4444
Epoch: 020, Train Acc: 0.4862, Test Acc: 0.5556
Epoch: 021, Train Acc: 0.4862, Test Acc:

## Triangles and Hexagons

In [108]:
dataset = torch.load('hexagon_812_final.pt')

In [109]:
for i in range(len(dataset)):
  a=pd.DataFrame(dataset[i].x.numpy()).drop([0,1],axis=1).to_numpy() #drop id
  dataset[i].x=torch.from_numpy(a)

In [110]:
import random
random.shuffle(dataset)

In [111]:
train_dataset = dataset[:650]
test_dataset = dataset[650:]

In [112]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [113]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(4, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(4, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [114]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.03)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x.float(), data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x.float(), data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5015, Test Acc: 0.5432
Epoch: 002, Train Acc: 0.4831, Test Acc: 0.5679
Epoch: 003, Train Acc: 0.5169, Test Acc: 0.4321
Epoch: 004, Train Acc: 0.4831, Test Acc: 0.5679
Epoch: 005, Train Acc: 0.5169, Test Acc: 0.4321
Epoch: 006, Train Acc: 0.5169, Test Acc: 0.4321
Epoch: 007, Train Acc: 0.5169, Test Acc: 0.4321
Epoch: 008, Train Acc: 0.5169, Test Acc: 0.4321
Epoch: 009, Train Acc: 0.5169, Test Acc: 0.4321
Epoch: 010, Train Acc: 0.5169, Test Acc: 0.4321
Epoch: 011, Train Acc: 0.5169, Test Acc: 0.4321
Epoch: 012, Train Acc: 0.5169, Test Acc: 0.4321
Epoch: 013, Train Acc: 0.5169, Test Acc: 0.4321
Epoch: 014, Train Acc: 0.5169, Test Acc: 0.4321
Epoch: 015, Train Acc: 0.5169, Test Acc: 0.4321
Epoch: 016, Train Acc: 0.4831, Test Acc: 0.5679
Epoch: 017, Train Acc: 0.5169, Test Acc: 0.4321
Epoch: 018, Train Acc: 0.5169, Test Acc: 0.4321
Epoch: 019, Train Acc: 0.5169, Test Acc: 0.4321
Epoch: 020, Train Acc: 0.5169, Test Acc: 0.4321
Epoch: 021, Train Acc: 0.4831, Test Acc: